In [12]:
from bs4 import BeautifulSoup as bs
import csv
import requests


In [13]:
cookies = {
    '_ga_D7VPPXYD0V': 'GS2.1.s1748523802$o1$g0$t1748523802$j60$l0$h0',
    '_ga': 'GA1.1.349693228.1748523804',
    'OptanonAlertBoxClosed': '2025-05-29T13:03:32.235Z',
    'minVersion': '{"experiment":-1458757693,"minFlavor":"domadaptermi-1.17.1.340.js100"}',
    '__eoi': 'ID=620426d8593f9773:T=1765207555:RT=1766141357:S=AA-Afja7_bvsbtJda5i1ne1TN_NC',
    'cebs': '1',
    '_ce.clock_data': '274%2C194.242.49.233%2C1%2C89db729cfcdc129111f017b0e7ac324a%2CChrome%2CUS',
    'shell#lang': 'en',
    'minUnifiedSessionToken10': '%7B%22sessionId%22%3A%22edb1dc4e1c-e237f7a625-2f81abf3c3-2d281e47c7-a16211a504%22%2C%22uid%22%3A%222b74b3443f-4f69173fe3-a1a2f70a46-d0456b1c0f-5b8b5f7047%22%2C%22__sidts__%22%3A1767371222008%2C%22__uidts__%22%3A1767371222008%7D',
    '__cf_bm': 'fKFdxnaILGJX5tQVFAldmvlWEGok8FoN5sMOhN4mQMg-1767375142-1.0.1.1-1Cvh.Yhramsg4yV5VPZkKFujh39BE21dxDjDPF.lcg.IxPqDqQ5.DxI0xxKEVqKctIz6w.gV8P8j0zIcpZ9FqhlEUGmxy4oZKrESPZjxa9g',
    'cf_clearance': 'hcNNNGNsHI7G_q0DbanzguU9JqpoSOTa5jlFlUBhB3I-1767375145-1.2.1.1-cfihV6Xb.4vEusx7dV_af2WoYf7XEF0yykRJ64uQPJkaTbgK1LjqHHHEzMiLcWHzx1V17yoFpUZ24TuryKo7W8udKyaCOUDoKwvAEGLWXkRbFqElEjvXb3og9T66z.bNstnKsCsKM0wm4.OX_cPmAj5chHFLlV3CRUMvixjfnfATO6JMXu5QvSlflkgZZAPR6X_X2gVqo7CwercdAMhxpfgrauzzh73kT.jLpNj8xaE',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Fri+Jan+02+2026+18%3A42%3A55+GMT%2B0100+(Ora+standard+dell%E2%80%99Europa+centrale)&version=202502.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=5277618f-a1ab-4f8f-878f-e1c64bad962f&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A0%2CC0004%3A0%2CC0003%3A0&intType=2&geolocation=IT%3B21&AwaitingReconsent=false',
    '_ce.s': 'v~27f75c0599e974665559df6723903953821b7bf8~lcw~1767375147623~vir~returning~lva~1767369960004~vpv~5~v11ls~02170600-e801-11f0-ae7f-21a17979315f~v11.cs~356004~v11.s~02170600-e801-11f0-ae7f-21a17979315f~v11.vs~27f75c0599e974665559df6723903953821b7bf8~v11.fsvd~eyJ1cmwiOiJhdHB0b3VyLmNvbS9lbi9yYW5raW5ncy9zaW5nbGVzIiwicmVmIjoiaHR0cHM6Ly93d3cuYXRwdG91ci5jb20vZW4vc2NvcmVzL2FyY2hpdmUvYnJpc2JhbmUvMzM5LzIwMjUvcmVzdWx0cyIsInV0bSI6W119~v11.sla~1767375147618~v11.wss~1767375147618~v11.ss~1767375147621~lcw~1767375776599',
    'cebsp_': '42',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'it-IT,it;q=0.9,en;q=0.8,en-US;q=0.7,es;q=0.6',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'referer': 'https://www.atptour.com/en/rankings/singles?dateWeek=2024-12-30&rankRange=101-200',
    'sec-ch-ua': '"Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
}

In [14]:
# build a list of dates for each week in 2023, 2024, 2025

from datetime import date, timedelta
start = date(2023, 1, 1)
# trova il primo lunedì del 2023
first_monday = start + timedelta(days=(7 - start.weekday()) % 7)
# una data per ogni settimana (copre 2023-2025)
weeks = [first_monday + timedelta(weeks=i) for i in range(157)]

In [15]:
def write_csv_senza_rip(filename, rows, mode='w'):
    """Write player rows to CSV. If mode=='w' writes header, 'a' appends without header."""
    # usiamo newline='' ed encoding per compatibilità cross-platform
    with open(filename, mode=mode, newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if mode == 'w':
            writer.writerow(["Player Id", "Player Name", "Nationality"])
        # inserire lista cose
        if rows:
            writer.writerows(rows)

In [16]:
# get weekly rankings for each week (separati per anno)
# Processiamo ogni anno separatamente in modo da mantenere set distinti
years = [2024, 2025]
tot_ranking_search = True  # Impostare a True per ottenere il ranking totale con ripetizioni

for year in years:
    print(f'Processing year {year}')
    player_ids_visti = set()  # Set per tenere traccia dei player_id già visti per questo anno
    lista_senza_ripetizioni = []  # Lista per memorizzare i player_id senza ripetizioni per questo anno
    
    lista_ranking_settimane = []

    # ciclare sulle settimane e processare solo quelle del 'year' corrente
    for w in weeks:
        if w.year != year:
            continue
        params = {
            'dateWeek': w,
            'rankRange': '0-5000',
        }
        response = requests.get('https://www.atptour.com/en/rankings/singles', params=params, cookies=cookies, headers=headers)
        print(response)

        soup = bs(response.text, 'html.parser')
        rows = soup.findAll('tr', class_='lower-row')[49:300]

        for l in rows:
            cols = l.findAll('td')
            player_a = cols[1].find('a')
            player_href = player_a['href']
            player_id = player_href.split('/')[4]

            player_rank = cols[0].get_text(strip=True)
            
            if tot_ranking_search ==False:
                if player_id not in player_ids_visti:
                    player_name = player_a.get_text(strip=True)
                    flag_use = cols[1].find('use')
                    player_national = flag_use['href'].split('#flag-')[1]
                    lista_senza_ripetizioni.append([player_id, player_name, player_national])
                    player_ids_visti.add(player_id)
            else:
                l = [w,player_id, player_rank]
                lista_ranking_settimane.append(l)
        
    if tot_ranking_search ==True:
        out_file = f"atp_ranking_settimanale_{year}.csv"
        # scriviamo in modalità 'w' per creare/riscrivere il file per l'anno
        write_csv_senza_rip(out_file, lista_ranking_settimane, mode='w')
        print(f'Wrote {len(lista_ranking_settimane)} weekly rankings for {year} to {out_file}')
        
    else:

        out_file = f"atp_senza_ripetizioni_{year}.csv"
        # scriviamo in modalità 'w' per creare/riscrivere il file per l'anno
        write_csv_senza_rip(out_file, lista_senza_ripetizioni, mode='w')
        print(f'Wrote {len(lista_senza_ripetizioni)} players for {year} to {out_file}')


Processing year 2024
<Response [200]>


/var/folders/kc/4bwqsb9s5556hjgp1jjmlg6h0000gn/T/ipykernel_75438/3962429644.py:25: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  rows = soup.findAll('tr', class_='lower-row')[49:300]
/var/folders/kc/4bwqsb9s5556hjgp1jjmlg6h0000gn/T/ipykernel_75438/3962429644.py:28: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  cols = l.findAll('td')


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Wrote 11044 weekly rankings for 2024 to atp_ranking_settimanale_2024.csv
Processing year 2025
<Response [200]>
<Resp